In [3]:
## Bibliotheken importieren
import pandas as pd

In [ ]:
## Top 1000-Liste von switch.ch importieren
directory = 'data/import/'
all_data = pd.DataFrame()

for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        date = filename[8:14]
        urls_import = pd.read_csv(directory + filename, header=0, names=["domainname", date])
        all_data = pd.concat([all_data, urls_import.set_index('domainname')], axis=1, sort=False)

In [54]:
## Datensatz transponieren
data_tp = all_data.T

In [59]:
## Datetimeindex erstellen
data_tp["date"] = pd.to_datetime(data_tp.index, format="%y%m")
data_tp = data_tp.set_index("date")

In [70]:
## zusammengefasste und formatierte Daten zwischenspeichern ##
data_tp.to_pickle('data/all_data.pkl')

In [4]:
## Und gleich wieder laden:
data_tp = pd.read_pickle('data/all_data.pkl')

In [5]:
# Zur Visualisierung in R bereitstellen:
data_tp.reset_index().to_feather('data/all_data.feather')

In [6]:
# Anzahl eindeutiger Domains ermitteln
len(data_tp.columns)

1467

### Meta-Daten zu URLs zur späteren Klassifizierung mittels Beautifulsoup ermitteln:

In [2]:
## Bibliotheken importieren
from bs4 import BeautifulSoup
import requests
import pprint
import sys

In [13]:
# urls_metadata = []
for website in data_tp.columns[747:]:
    url_metadata = []
    url_metadata.append(website)
    url = "http://www.{}".format(website)
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text)
    
        metas = soup.find_all('meta')
        title = soup.find('title')
        if (title != None):
            title = title.text
        url_metadata.append(title)
        for meta in metas:
            if ('name' in meta.attrs and meta.attrs['name'] == 'description' and 'content' in meta.attrs):
                description = meta.attrs['content']
                url_metadata.append(description)
            elif ('name' in meta.attrs and meta.attrs['name'] == 'keywords' and 'content' in meta.attrs):
                keywords = meta.attrs['content']
                url_metadata.append(keywords)
                
    except requests.exceptions.RequestException as e:
        print(e)
            
    urls_metadata.append(url_metadata)

HTTPSConnectionPool(host='www.netgrouper.ch', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'ssl3_read_bytes', 'sslv3 alert handshake failure')])")))
HTTPConnectionPool(host='www.messaging.ch', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x11c299278>: Failed to establish a new connection: [Errno 60] Operation timed out'))
HTTPConnectionPool(host='www.swissprivacylab.ch', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x11bb17358>: Failed to establish a new connection: [Errno 51] Network is unreachable'))
HTTPConnectionPool(host='www.pilatus-aircraft.ch', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x11ba5c898>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known

adidas.ch bei index 746 führt zu einem Hänger in der Schleife - deshalb wurde es übersprungen.

In [31]:
## Metadaten zwischenspeichern ##
pd.DataFrame.from_records(urls_metadata, columns=["url", "title", "description", "keywords", "NaN", "NaN"]).to_pickle('data/metadata.pkl')


In [59]:
## Und gleich wieder laden:
urls_metadata_df = pd.read_pickle('data/metadata.pkl')

### Offene Ports der einzelnen Domains ermitteln (um allfällige andere Aufgaben hinter den Domains zu finden):

In [94]:
## Bibliotheken importieren
import nmap
import numpy as np
nm = nmap.PortScanner()

In [51]:
## Portscanner Test
scanresult = nm.scan('tagesanzeiger.ch', arguments='-Pn')

In [219]:
## URLs und ihre offenen Ports (die via Droplet ermittelt wurden) einlesen:
url_ports = pd.read_pickle('data/url_ports.pkl')

In [291]:
## Domains ohne Webserver ermitteln
for website in url_ports[(url_ports["80-http"] != "open") & (url_ports["443-https"] != "open")].url:
    line = url_ports[url_ports["url"] == website].T
    print("{} - {}".format(website, line.index[line.iloc[:,0] == "open"].values))

comlaude.ch - []
hostfactory.ch - ['25-smtp']
dfinet.ch - []
datazug.ch - []
syhosting.ch - []
trendhosting-net.ch - ['22-ssh' '25-smtp' '53-domain']
pcdog.ch - []
datacomm.ch - []
sinkhole.ch - ['22-ssh']
oriented.ch - []
scalera.ch - []
pimpernet.ch - []
oca.ch - ['53-domain']
nameserver.ch - ['25-smtp' '110-pop3' '143-imap' '587-submission']
intername.ch - []
lu.ch - ['53-domain']
movie4u.ch - ['22-ssh']
novahosting.ch - []
loterie.ch - []
q-x.ch - []
com.ch - []
ntb.ch - ['53-domain']
swissprivacylab.ch - []
dynamic-net.ch - ['25-smtp' '21-ftp' '53-domain' '110-pop3' '143-imap' '109-pop2'
 '1984-bigbrother']
ebcomdns.ch - []
weaver.ch - []
goline.ch - ['53-domain']
prevosti.ch - ['1024-kdm']
clinch.ch - ['25-smtp' '21-ftp' '110-pop3']
host-n2o.ch - []
fastbooking.ch - []
fasthosting.ch - []
oekohosting.ch - []
tilllate.ch - []
ibone.ch - ['25-smtp' '21-ftp' '53-domain' '110-pop3' '143-imap' '109-pop2'
 '1984-bigbrother']
eds.ch - []
spectrospin.ch - ['21-ftp' '264-bgmp']
psinet.ch 

In [250]:
url_ports

,url,ip,80-http,443-https,22-ssh,43-whois,8008-http,8080-http-proxy,8443-https-alt,113-ident,...,49165-unknown,49167-unknown,49175-unknown,49400-compaqdiag,49999-unknown,55055-unknown,55056-unknown,56737-unknown,57294-unknown,57797-unknown
0,switch.ch,130.59.31.80,open,open,,,,,,,...,,,,,,,,,,
1,finarea.ch,77.72.174.19,open,open,,,,,,,...,,,,,,,,,,
2,comlaude.ch,213.212.81.71,,,,,,,,,...,,,,,,,,,,
3,infomaniak.ch,84.16.73.14,open,open,open,open,closed,closed,closed,,...,,,,,,,,,,
4,noris.ch,62.128.1.30,open,open,,,,,,closed,...,,,,,,,,,,
5,google.ch,172.217.18.99,open,open,,,,,,,...,,,,,,,,,,
6,hostpoint.ch,217.26.48.199,open,open,open,,,,,,...,,,,,,,,,,
7,honcode.ch,205.178.189.129,open,,,,,,,closed,...,,,,,,,,,,
8,cern.ch,188.184.9.234,open,open,,,,,,,...,,,,,,,,,,
9,bluewin.ch,213.3.75.34,open,open,,,,,,,...,,,,,,,,,,


### Domains mit Hilfe von externer Webseite klassifizieren:

In [5]:
## Bibliotheken importieren
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import json
import requests
import time
import re
from pathlib import Path

In [40]:
## Webseiten Mithilfe von Bluecoat Sitereview klassifizieren
driver = webdriver.Chrome("./tools/chromedriver")
url = "https://sitereview.bluecoat.com/"
url_categories = []
driver.get(url)
driver.implicitly_wait(100)
for url in data_tp.columns[1226:]:
    url_entry = [url]
    time.sleep(1)
    driver.find_element_by_id("txtSearch").send_keys(url)
    time.sleep(1)
    driver.find_element_by_id("btnLookupSubmit").click()
    time.sleep(1)
    category_entry = []
    categories = driver.find_elements_by_class_name("clickable-category")
    if categories:
        for category in categories:
            category_entry.append(category.text)
    url_entry.append(category_entry)
    url_categories.append(url_entry)
    try:
        driver.find_element_by_xpath("/html[1]/body[1]/app-root[1]/div[1]/section[1]/div[1]/ng-component[1]/form[1]/p[1]/a[1]").click()
    except:
        driver.find_element_by_id("btnOk").click()
        driver.find_element_by_xpath("/html[1]/body[1]/app-root[1]/div[1]/section[1]/div[1]/ng-component[1]/form[1]/p[1]/a[1]").click()
driver.quit()

In [48]:
## DF erstellen
url_categories_df = pd.DataFrame(columns=['url', 'n_categories'])
for url in url_categories:
    n_categories = 0
    if (len(url[1]) > 0):
        for idx, category in enumerate(url[1]):
            if category not in url_categories_df.columns.values:
                url_categories_df[category] = ""
            n_categories = idx + 1
    x = pd.DataFrame([[False] * len(url_categories_df.columns.values)], columns = url_categories_df.columns.values)
    x['url'] = url[0]
    x['n_categories'] = n_categories
    if len(url[1]) > 0:
        for idx, category in enumerate(url[1]):
            x[category] = True
    url_categories_df = url_categories_df.append(x)

In [52]:
## URL-Kategorien zwischenspeichern ##
url_categories_df.to_pickle('data/url_categories.pkl')

In [53]:
# und URL-Kategorien gleich wieder einlesen:
url_categories_df = pd.read_pickle('data/url_categories.pkl')

In [85]:
## Blick in die Daten
url_categories_df.url[url_categories_df['Bösartige ausgehende Daten/Botnets (Malicious Outbound Data/Botnets)'] == True]


0    sinkhole.ch
0     default.ch
Name: url, dtype: object

In [55]:
## Anzahl Kategorien ermitteln 
len(url_categories_df.columns) - 2

61

In [58]:
## Durchschnittswerte pro Kategorie ermitteln (Beliebtheit pro Kategorie)
from datetime import datetime
import numpy as np

categories_data = pd.DataFrame(columns=["Name", "n_urls", "avg_1901", "avg_1812", "avg_1811", "avg_1810", "avg_1809", "avg_1808", "avg_1807", "avg_1806", "avg_1805", "avg_1804", "avg_1803", "avg_1802", "avg_1801", "avg_1712", "avg_1711", "avg_1710", "avg_1709", "avg_1708", "avg_1707"])
for cat in url_categories_df.columns.values[2:]:
    visits = {
        "avg_1901": 0, 
        "avg_1812": 0,
        "avg_1811": 0,
        "avg_1810": 0,
        "avg_1809": 0,
        "avg_1808": 0,
        "avg_1807": 0,
        "avg_1806": 0,
        "avg_1805": 0,
        "avg_1804": 0,
        "avg_1803": 0,
        "avg_1802": 0,
        "avg_1801": 0,
        "avg_1712": 0,
        "avg_1711": 0,
        "avg_1710": 0,
        "avg_1709": 0,
        "avg_1708": 0,
        "avg_1707": 0
    }

    for website in (url_categories_df.url[url_categories_df[cat] == True]):
        for average in visits:
            visits[average] += 0 if np.isnan(data_tp[website][datetime.strftime(datetime.strptime(average[4:], '%y%m'), "%Y-%m-%d")][0]) else data_tp[website][datetime.strftime(datetime.strptime(average[4:], '%y%m'), "%Y-%m-%d")][0]
    for average in visits:
        visits[average] = visits[average]/url_categories_df.url[url_categories_df[cat] == True].count()
    visits["Name"] = cat
    visits["n_urls"] = url_categories_df.url[url_categories_df[cat] == True].count()
    categories_data = categories_data.append(visits, ignore_index=True)



In [59]:
## Metadaten zu Kategorien zwischenspeichern und für Visualisierung in R exportieren ##
categories_data.to_pickle('data/categories_data.pkl')

In [60]:
# und URL-Kategorien gleich wieder einlesen:
categories_data = pd.read_pickle('data/categories_data.pkl')

---

In [81]:
## duchschnittliche Besuchswerte pro Kategorie ermitteln
categories_data[categories_data["Name"] == "Phishing (Phishing)"]

,Name,n_urls,avg_1901,avg_1812,avg_1811,avg_1810,avg_1809,avg_1808,avg_1807,avg_1806,...,avg_1804,avg_1803,avg_1802,avg_1801,avg_1712,avg_1711,avg_1710,avg_1709,avg_1708,avg_1707
56,Phishing (Phishing),3,9592.0,8760.333333,27328.333333,32579.666667,43638.333333,31631.333333,33033.666667,34754.333333,...,35439.333333,36389.666667,34409.666667,37451.666667,35292.666667,34227.333333,33620.333333,32725.666667,30823.0,29327.666667


In [82]:
### Domains mit grossen Veränderungen in den DNS-Anfrage von einem Monat zum nächsten ermitteln
big_changes = []
for website in data_tp.columns:
    comparison = [0]
    for key, value in data_tp[website].iteritems():
        comparison.append(value)
        if (comparison[-1] != 0) and (comparison[-2] != 0):
            change = (100/comparison[-2]*comparison[-1]) - 100
            if abs(change) > 60:
                big_changes.append([key.strftime('%Y-%m'), website, change])

In [83]:
### Blick in die Daten
big_changes

[['2018-11', 'kbntx.ch', -62.460696311701774],
 ['2018-08', 'kbntx.ch', -70.58729965662104],
 ['2018-01', 'adslplus.ch', 202.3618577597603],
 ['2018-07', 'datasource.ch', 129.73067955259688],
 ['2018-03', 'dns-net.ch', 69.94181663541647],
 ['2018-12', 'cablecom.ch', 131.86579018739263],
 ['2017-11', 'cablecom.ch', 102.79897163708742],
 ['2017-09', 'cablecom.ch', 101.02704141656147],
 ['2017-08', 'cablecom.ch', -70.08645951695443],
 ['2017-07', 'cablecom.ch', 109.2707098690505],
 ['2018-05', 'senselan.ch', 131.6181815590302],
 ['2017-08', 'ggaweb.ch', 75.49254603916984],
 ['2018-12', 'softronics.ch', 60.851379024879094],
 ['2017-11', 'zhdk.ch', 82.80593462982932],
 ['2018-07', 'kissasian.ch', 68.24775104377395],
 ['2018-09', 'hefr.ch', 167.78766533442268],
 ['2018-04', 'kesako.ch', 88.66171968242438],
 ['2018-03', 'domrobot.ch', 80.83135502490342],
 ['2017-08', 'kuoni.ch', 82.79231739601656],
 ['2017-08', '4server.ch', 80.15663597574667],
 ['2017-10', 'orange.ch', 69.51104905619849],
 [

In [84]:
### Bösartige URLs ermitteln
url_categories_df.url[(url_categories_df['Bösartige ausgehende Daten/Botnets (Malicious Outbound Data/Botnets)'] == True) |
                     (url_categories_df['Phishing (Phishing)'] == True) | 
                     (url_categories_df['Spam (Spam)'] == True) |
                     (url_categories_df['Bösartige Quellen/Malnets (Malicious Sources/Malnets)'] == True) |
                     (url_categories_df['Verdächtig (Suspicious)'] == True) |
                     (url_categories_df['Raubkopien/Urheberrechtsverletzung (Piracy/Copyright Concerns)'] == True) |
                     (url_categories_df['Möglicherweise unerwünschte Software (Potentially Unwanted Software)'] == True)]


0             arenabg.ch
0             zer0day.ch
0            adslplus.ch
0              gotdns.ch
0     torrentdownload.ch
0            sinkhole.ch
0             square7.ch
0               zoink.ch
0          newasiantv.ch
0     watchfreemovies.ch
0           kissasian.ch
0       tamilrockerss.ch
0             movie4u.ch
0         cokepopcorn.ch
0             mpdkim2.ch
0              cyhost.ch
0             default.ch
0          mnbsserver.ch
0                eztv.ch
0             levidia.ch
0         ebookhunter.ch
0              wootly.ch
0                ceoo.ch
0         animeultima.ch
0            safesear.ch
0           naefmarco.ch
0                spee.ch
0                snd1.ch
0       ilcorsaronero.ch
0            1channel.ch
0            gurochan.ch
0              vidics.ch
0             todaypk.ch
0              planet.ch
0        ghielmimport.ch
0               pubmt.ch
0               tlaun.ch
0     project-free-tv.ch
0               usatv.ch
0    kellerinformatik.ch
